# Grab Batter Info

## Monday, November 7
## Sam LaFell

### Purpose:
This notebook is to evaluate batter information. We wnat to pull in various batter metrics.
- Metrics could include:
    - WAR
    - PA
    - Expected Batting Average
    - BABIP
    - Expected Slugging Pct
    - Etc Etc

In [150]:
import pybaseball as pb
import pandas as pd

# Set max cols
pd.set_option('display.max_columns', 350)
pd.set_option('display.max_rows', 100)

## Data Needed:
-  Player ID
- Plate Appearances
- expected Batting Average


## First Up: Player ID
### Start with One Team, One Game

In [151]:
# batting stats by year, regular season
from pybaseball.lahman import *
batting = batting()

In [152]:
batting.head(10)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,2,2,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,3,0,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0
5,armstbo01,1871,1,FW1,NaN,12,49,9,11,2,1,0,5.0,0.0,1.0,0,1.0,NaN,NaN,NaN,NaN,0.0
6,barkeal01,1871,1,RC1,NaN,1,4,0,1,0,0,0,2.0,0.0,0.0,1,0.0,NaN,NaN,NaN,NaN,0.0
7,barnero01,1871,1,BS1,NaN,31,157,66,63,10,9,0,34.0,11.0,6.0,13,1.0,NaN,NaN,NaN,NaN,1.0
8,barrebi01,1871,1,FW1,NaN,1,5,1,1,1,0,0,1.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
9,barrofr01,1871,1,BS1,NaN,18,86,13,13,2,1,0,11.0,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0


In [153]:
# SCD2 type table with start and end dates for each player at each team
grouped = batting.groupby(['playerID', 'teamID'])['yearID'].agg([min, max]).reset_index()
grouped.rename(columns={'min':'start', 'max':'end'})

,playerID,teamID,start,end
0,aardsda01,ATL,2015,2015
1,aardsda01,BOS,2008,2008
2,aardsda01,CHA,2007,2007
3,aardsda01,CHN,2006,2006
4,aardsda01,NYA,2012,2012
...,...,...,...,...
48776,zuverge01,DET,1954,1955
48777,zwilldu01,CHA,1910,1910
48778,zwilldu01,CHF,1914,1915
48779,zwilldu01,CHN,1916,1916


In [154]:
from pybaseball import batting_stats
from datetime import datetime

game_date = '2018-05-13'
game_year = datetime.strptime(game_date, '%Y-%m-%d').year

# get all of this season's batting data so far
data = batting_stats(game_year, qual=250)

# Team we want
team = 'ATL'
atl_batters = data.loc[data['Team']==team]

In [155]:
# Get Team Batters
atl_batters_lookup = pd.DataFrame()
atl_batter_list = atl_batters['Name'].values
for batter in atl_batter_list:
    last_name = batter.split(' ')[1]
    first_name = batter.split(' ')[0]
    batter_lookup = pb.playerid_lookup(last_name, first_name)
    atl_batters_lookup = pd.concat([atl_batters_lookup, batter_lookup])

In [156]:
# No team information in statcast data, can't limit to one team using this method, need the above batting_stats call to grab team info
from pybaseball import statcast_batter
atl_batters_playerids = atl_batters_lookup['key_mlbam'].values
statcast_batter(game_date, game_date, atl_batters_playerids[0])

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SI,2018-05-13,95.9,-1.91,6.04,"Freeman, Freddie",518692,570632,field_out,hit_into_play,NaN,NaN,NaN,NaN,4,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,X,6.0,ground_ball,1,2,2018,-1.18,0.74,-0.82,2.27,NaN,660670.0,NaN,0,6,Top,142.83,147.65,NaN,NaN,592663,NaN,NaN,5.547648,-139.511436,-6.010186,-16.584430,32.226430,-21.407928,3.54,1.60,14.0,89.1,-8.0,95.7,2025,6.3,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.23,0.200,0.191,0.0,1.0,0.0,0.0,2.0,40,4,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,238,0.007,-0.143
1,SL,2018-05-13,86.1,-2.10,6.03,"Freeman, Freddie",518692,570632,NaN,called_strike,NaN,NaN,NaN,NaN,7,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,S,NaN,NaN,1,1,2018,0.03,0.87,-0.32,1.58,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,4.112260,-125.276668,-6.041599,-0.480732,24.489871,-22.145530,3.69,1.68,NaN,NaN,NaN,85.6,2242,5.9,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,3,Slider,0,1,1,0,1,0,1,0,Infield shift,Standard,178,0.000,-0.079
2,SI,2018-05-13,93.7,-2.09,6.02,"Freeman, Freddie",518692,570632,NaN,foul,NaN,NaN,NaN,NaN,5,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,S,NaN,NaN,1,0,2018,-1.37,0.90,0.19,2.53,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,8.740775,-136.043112,-5.138068,-18.565569,28.733516,-20.363898,3.54,1.60,NaN,NaN,NaN,92.8,1932,5.8,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,2,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,237,0.000,-0.082
3,SI,2018-05-13,93.2,-2.14,6.07,"Freeman, Freddie",518692,570632,NaN,ball,NaN,NaN,NaN,NaN,11,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,B,NaN,NaN,0,0,2018,-1.37,0.78,-1.28,3.21,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,5.050969,-135.529980,-3.194410,-17.536910,29.239857,-22.356911,3.76,1.79,NaN,NaN,NaN,91.9,1926,5.5,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,1,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,240,0.000,0.045
4,SI,2018-05-13,94.3,-1.77,5.91,"Freeman, Freddie",518692,570632,walk,ball,NaN,NaN,NaN,NaN,13,Freddie Freeman walks. Ronald Acuna to 2nd.,R,L,R,MIA,ATL,B,NaN,NaN,3,0,2018,-1.29,0.94,-0.28,1.23,NaN,NaN,660670.0,2,3,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,6.702716,-136.968794,-8.458077,-17.659943,30.727316,-18.802751,3.74,1.85,NaN,NaN,NaN,93.9,2070,6.2,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.32,NaN,NaN,0.7,1.0,0.0,0.0,NaN,19,4,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,234,-0.019,0.073
5,SL,2018-05-13,89.2,-1.90,5.96,"Freeman, Freddie",518692,570632,NaN,ball,NaN,NaN,NaN,NaN,11,Freddie Freeman walks. Ronald Acuna to 2nd.,R,L,R,MIA,ATL,B,NaN,NaN,2,0,2018,-0.79,1.34,-1.75,2.97,NaN,NaN,660670.0,2,3,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,2.025415,-129.737625,-4.041573,-9.230952,31.362924

In [157]:
atl_batters_lookup['key_mlbam'].values

array([518692, 645277, 542255, 622666, 455976, 452095, 621020, 518586,
       435559])

## Turn these into functions

1. First step, we want to identify a specific game

In [158]:
statcast()

start_dt 2023-05-01
end_dt 2023-05-02
This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:07<00:00,  3.94s/it]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
835,FC,2023-05-01,92.5,-2.8,5.69,"Robertson, David",644433,502085,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Chadwick Tromp pops out to catcher Francisco A...,R,R,R,NYM,ATL,X,2,popup,1,0,2023,0.3,1.37,0.23,2.44,<NA>,645277,<NA>,2,9,Top,129.03,196.31,<NA>,<NA>,682626,<NA>,<NA>,7.133215,-134.432962,-5.497106,2.047296,29.153281,-14.603853,3.34,1.59,8,87.7,82,93.2,2670,6.8,718361,502085,682626,624413,643446,641645,596019,592192,607043,516782,53.73,0.001,0.0,0.0,1,0,0,3,72,2,Cutter,5,3,3,5,3,5,3,5,Standard,Standard,192,0.037,-0.332
856,KC,2023-05-01,85.2,-2.78,5.65,"Robertson, David",644433,502085,NaN,ball,<NA>,<NA>,<NA>,<NA>,14,Chadwick Tromp pops out to catcher Francisco A...,R,R,R,NYM,ATL,B,<NA>,NaN,0,0,2023,0.41,-0.7,0.31,1.0,<NA>,645277,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,682626,<NA>,<NA>,6.470021,-123.854384,-3.177414,2.836951,26.633264,-38.878741,3.35,1.59,<NA>,<NA>,<NA>,85.6,2745,6.7,718361,502085,682626,624413,643446,641645,596019,592192,607043,516782,53.76,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,72,1,Knuckle Curve,5,3,3,5,3,5,3,5,Standard,Standard,39,0.0,0.015
872,KC,2023-05-01,84.5,-2.85,5.74,"Robertson, David",671739,502085,strikeout,foul_tip,<NA>,<NA>,<NA>,<NA>,13,Michael Harris II strikes out on a foul tip.,R,L,R,NYM,ATL,S,2,NaN,1,2,2023,0.65,-1.11,-0.95,2.35,<NA>,645277,<NA>,1,9,Top,<NA>,<NA>,<NA>,<NA>,682626,<NA>,<NA>,3.145888,-122.979354,0.753362,5.819279,26.942378,-43.794015,3.48,1.64,<NA>,<NA>,<NA>,84.6,2862,6.6,718361,502085,682626,624413,643446,641645,596019,592192,607043,516782,53.93,<NA>,<NA>,0.0,1,0,0,<NA>,71,4,Knuckle Curve,5,3,3,5,3,5,3,5,Standard,Standard,45,0.053,-0.237
884,KC,2023-05-01,83.7,-2.97,5.55,"Robertson, David",671739,502085,NaN,ball,<NA>,<NA>,<NA>,<NA>,13,Michael Harris II strikes out on a foul tip.,R,L,R,NYM,ATL,B,<NA>,NaN,0,2,2023,0.92,-1.07,-1.63,1.76,<NA>,645277,<NA>,1,9,Top,<NA>,<NA>,<NA>,<NA>,682626,<NA>,<NA>,1.27808,-121.783274,-0.117158,8.808708,27.198797,-42.911994,3.55,1.64,<NA>,<NA>,<NA>,83.7,2848,6.6,718361,502085,682626,624413,643446,641645,596019,592192,607043,516782,53.9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,71,3,Knuckle Curve,5,3,3,5,3,5,3,5,Standard,Standard,52,0.0,0.036
911,KC,2023-05-01,84.0,-2.89,5.67,"Robertson, David",671739,502085,NaN,foul,<NA>,<NA>,<NA>,<NA>,13,Michael Harris II strikes out on a foul tip.,R,L,R,NYM,ATL,S,<NA>,NaN,0,1,2023,0.89,-0.89,-0.36,1.07,<NA>,645277,<NA>,1,9,Top,<NA>,<NA>,<NA>,<NA>,682626,<NA>,<NA>,4.107478,-122.328254,-2.487487,8.024385,25.631081,-40.740938,3.48,1.64,1,47.8,-37,84.4,2639,6.6,718361,502085,682626,624413,643446,641645,596019,592192,607043,516782,53.85,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,71,2,Knuckle Curve,5,3,3,5,3,5,3,5,Strategic,Standard,46,0.0,-0.117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [159]:
from pybaseball import statcast
import pandas as pd

def gather_game_info(game_day: str, team: str) -> pd.DataFrame:
    """
    Get Statcast data for a given game and team.

    Args:
        game_day (str): Date of the game in 'YYYY-MM-DD' format.
        team (str): Three-letter abbreviation for the team.

    Returns:
        pd.DataFrame: A DataFrame containing Statcast data for the game and team.

    Raises:
        ValueError: If no data is found for the given game and team.
    """
    # Statcast Info
    data = statcast(start_dt=game_day, end_dt=game_day, team=team)

    # No game for the team on this date
    if data.shape[0] == 0:
        raise ValueError(f"No data found for game on {game_day} for team {team}")

     # Get new team
    team_list = data[['home_team', 'away_team']].drop_duplicates()
    new_team = [new_team for new_team in team_list.values[0] if new_team != team]
    
    # Get Statcast info for the 2nd team
    data2 = statcast(start_dt=game_day, end_dt=game_day, team=new_team[0])

    return pd.concat([data, data2])

In [160]:
game_day = '2023-04-15'
data = gather_game_info(game_day, 'LAD')
data2 = gather_game_info(game_day, 'CHC')

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:01<00:00,  1.03s/it]

This is a large query, it may take a moment to complete



100%|██████████| 1/1 [00:01<00:00,  1.28s/it]

This is a large query, it may take a moment to complete



100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

This is a large query, it may take a moment to complete



100%|██████████| 1/1 [00:01<00:00,  1.55s/it]


# Data Check

In [161]:
assert data['batter'].drop_duplicates().shape[0] >= 18, 'Not enough batters recognized between the two teams for one of these games. Why?'

## Track Batters in original game over last 4 weeks

In [162]:
batters = data['batter'].drop_duplicates().values

from pybaseball import playerid_reverse_lookup
playerid_reverse_lookup(batters, key_type = 'mlbam')

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,wisdom,patrick,621550,wisdp001,wisdopa01,13602,2018.0,2023.0
1,hoerner,nico,663538,hoern001,hoernni01,21479,2019.0,2023.0
2,freeman,freddie,518692,freef001,freemfr01,5361,2010.0,2023.0
3,rojas,miguel,500743,rojam002,rojasmi02,7802,2014.0,2023.0
4,bellinger,cody,641355,bellc002,bellico01,15998,2017.0,2023.0
5,outman,james,681546,outmj002,outmaja01,24770,2022.0,2023.0
6,suzuki,seiya,673548,suzus001,suzukse01,30116,2022.0,2023.0
7,muncy,max,571970,muncm001,muncyma01,13301,2015.0,2023.0
8,rios,edwin,621458,riose001,riosed01,18316,2019.0,2023.0
9,heyward,jason,518792,heywj001,heywaja01,4940,2010.0,2023.0


In [163]:
from pybaseball import statcast_batter

# If Game Date is after 5/1, do this
from datetime import timedelta, datetime
first_day_range = (datetime.strptime(game_day, '%Y-%m-%d') - timedelta(days = 21)).strftime('%Y-%m-%d')
last_day_range = (datetime.strptime(game_day, '%Y-%m-%d') - timedelta(days = 1)).strftime('%Y-%m-%d')

batter_performance_df = pd.DataFrame()
for batter in batters:
    batter_performance = statcast_batter(start_dt=first_day_range, end_dt=last_day_range, player_id=batter)
    batter_performance_df = pd.concat([batter_performance_df, batter_performance])

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [164]:
avg_bas_last_3weeks = pd.DataFrame(batter_performance_df.groupby('batter')['estimated_ba_using_speedangle'].mean()).reset_index()
avg_bas_last_3weeks

,batter,estimated_ba_using_speedangle
0,444482,0.336931
1,500743,0.250000
2,502110,0.448756
3,518692,0.437463
4,518792,0.390929
5,543333,0.265321
6,571466,0.382364
7,571970,0.408333
8,605131,0.143071
9,605141,0.368545


In [165]:
avg_bas_last_3weeks.pivot_table(columns='batter', values='estimated_ba_using_speedangle')

batter,444482,500743,502110,518692,518792,543333,571466,571970,605131,605141,621020,621458,621550,641355,641820,663538,664023,673548,678246,681546
estimated_ba_using_speedangle,0.336931,0.25,0.448756,0.437463,0.390929,0.265321,0.382364,0.408333,0.143071,0.368545,0.423656,0.321286,0.36975,0.290475,0.349844,0.348404,0.388138,0.30875,0.349462,0.451074


In [166]:
batter_performance['estimated_ba_using_speedangle'].mean()

0.25000000000000006

## New Direction
- So, up till now, the columns don't make sense, they need to be more generalized.
- So, creating columns that are home_1-home_9, and the same for road hitters.
- This will take the most likely starter at each position and record their batting stats over the last 3 weeks.
- So, for example, home_1 will be the pitchers info or the DH (if AL).

In [167]:
batter_performance.groupby(['batter', 'game_date'])['at_bat_number'].nunique()

batter  game_date 
500743  2023-03-26    3
        2023-03-27    2
        2023-03-28    3
        2023-03-30    4
        2023-03-31    4
        2023-04-01    4
        2023-04-02    3
        2023-04-09    4
        2023-04-11    3
        2023-04-12    2
        2023-04-14    3
Name: at_bat_number, dtype: int64

In [168]:
position_dict = {
 'P': 1,
 'C': 2,
 '1B': 3,
 '2B': 4,
 '3B': 5,
 'SS': 6,
 'LF': 7,
 'CF': 8,
 'RF': 9,
 'DH': 10
}

In [169]:
# What about position?
from pybaseball import fielding_leaders
fielding_df = fielding_leaders.fg_fielding_data(start_season = 2019, end_season = 2023, split_seasons = 1)[['IDfg', 'Season', 'Name', 'Team', 'Pos', 'G', 'GS']]
fielding_df[['First Name', 'Last Name']] = fielding_df['Name'].str.split(' ', n=1, expand=True)
fielding_df['Rank'] = fielding_df.groupby(['Season', 'Team', 'Pos'])['GS'].rank(ascending=False, method='dense')
fielding_df['Pos_ID'] = fielding_df['Pos'].map(position_dict)

In [170]:
fielding_df

,IDfg,Season,Name,Team,Pos,G,GS,First Name,Last Name,Rank,Pos_ID
0,12979,2019,Javier Baez,CHC,SS,129,128,Javier,Baez,1.0,6
1,19339,2021,Nicky Lopez,KCR,SS,148,141,Nicky,Lopez,1.0,6
2,11739,2019,J.T. Realmuto,PHI,C,133,130,J.T.,Realmuto,1.0,2
3,12916,2019,Francisco Lindor,CLE,SS,137,137,Francisco,Lindor,1.0,6
4,11368,2019,Yasmani Grandal,MIL,C,137,124,Yasmani,Grandal,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
555,11737,2022,Nick Castellanos,PHI,RF,121,119,Nick,Castellanos,1.0,9
556,20123,2022,Juan Soto,- - -,RF,151,151,Juan,Soto,1.0,9
557,3516,2019,Eric Hosmer,SDP,1B,157,155,Eric,Hosmer,1.0,3
558,19566,2022,Nathaniel Lowe,TEX,1B,153,150,Nathaniel,Lowe,1.0,3


In [175]:
from pybaseball import playerid_lookup

player_id_df = pd.DataFrame()
for i in range(len(fielding_df)):
    last_name = fielding_df.iloc[i, [-4,-3]].values[1]
    first_name = fielding_df.iloc[i, [-4,-3]].values[0]
    player_id_lookup = playerid_lookup(last = last_name, first = first_name, fuzzy = True)
    player_id_df = pd.concat([player_id_df, player_id_lookup])
    
player_id_df = player_id_df.drop_duplicates()

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

In [178]:
from unidecode import unidecode

# Define a function to remove accents from a string
def remove_accents(s):
    return unidecode(s)

# Apply the remove_accents function to the name_last column
player_id_df['name_last'] = player_id_df['name_last'].apply(remove_accents)

In [182]:
fielding_df

,IDfg,Season,Name,Team,Pos,G,GS,First Name,Last Name,Rank,Pos_ID
0,12979,2019,Javier Baez,CHC,SS,129,128,Javier,Baez,1.0,6
1,19339,2021,Nicky Lopez,KCR,SS,148,141,Nicky,Lopez,1.0,6
2,11739,2019,J.T. Realmuto,PHI,C,133,130,J.T.,Realmuto,1.0,2
3,12916,2019,Francisco Lindor,CLE,SS,137,137,Francisco,Lindor,1.0,6
4,11368,2019,Yasmani Grandal,MIL,C,137,124,Yasmani,Grandal,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
555,11737,2022,Nick Castellanos,PHI,RF,121,119,Nick,Castellanos,1.0,9
556,20123,2022,Juan Soto,- - -,RF,151,151,Juan,Soto,1.0,9
557,3516,2019,Eric Hosmer,SDP,1B,157,155,Eric,Hosmer,1.0,3
558,19566,2022,Nathaniel Lowe,TEX,1B,153,150,Nathaniel,Lowe,1.0,3


In [194]:
# Convert the 'First Name' and 'Last Name' columns to lowercase
fielding_df[['First Name', 'Last Name']] = fielding_df[['First Name', 'Last Name']].apply(lambda x: x.str.lower())
player_id_df[['name_first', 'name_last']] = player_id_df[['name_first', 'name_last']].apply(lambda x: x.str.lower())

# Merge the DataFrames on lowercase 'First Name' and 'Last Name'
merged_df = pd.merge(fielding_df, player_id_df, left_on=['First Name', 'Last Name'], right_on=['name_first', 'name_last'])

# Drop redundant columns
merged_df.drop(['First Name', 'Last Name'], axis=1, inplace=True)

# Show
merged_df

,IDfg,Season,Name,Team,Pos,G,GS,Rank,Pos_ID,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,12979,2019,Javier Baez,CHC,SS,129,128,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
1,12979,2020,Javier Baez,CHC,SS,56,56,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
2,12979,2022,Javier Baez,DET,SS,133,132,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
3,12979,2023,Javier Baez,DET,SS,23,22,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
4,19339,2021,Nicky Lopez,KCR,SS,148,141,1.0,6,lopez,nicky,670032,lopen001,lopezni01,19339,2019.0,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,9847,2021,Andrew McCutchen,PHI,LF,135,131,1.0,7,mccutchen,andrew,457705,mccua001,mccutan01,9847,2009.0,2023.0
509,6368,2019,Adam Jones,ARI,RF,130,117,1.0,9,jones,adam,430945,jonea003,jonesad01,6368,2006.0,2019.0
510,14274,2021,Mitch Haniger,SEA,RF,123,122,1.0,9,haniger,mitch,571745,hanim001,hanigmi01,14274,2016.0,2023.0
511,13590,2022,Jesse Winker,SEA,LF,118,113,1.0,7,winker,jesse,608385,winkj002,winkeje01,13590,2017.0,2023.0


In [195]:
merged_df[merged_df['Name'] == 'Javier Baez']

,IDfg,Season,Name,Team,Pos,G,GS,Rank,Pos_ID,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,12979,2019,Javier Baez,CHC,SS,129,128,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
1,12979,2020,Javier Baez,CHC,SS,56,56,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
2,12979,2022,Javier Baez,DET,SS,133,132,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0
3,12979,2023,Javier Baez,DET,SS,23,22,1.0,6,baez,javier,595879,baezj001,baezja01,12979,2014.0,2023.0


In [201]:
# Error in when a player plays for more than 2 teams in 1 season
fielding_df2 = fielding_leaders.fg_fielding_data(start_season = 2021)[['IDfg', 'Season', 'Name', 'Team', 'Pos', 'G', 'GS']]
fielding_df2[fielding_df2['Name'] == 'Javier Baez']

,IDfg,Season,Name,Team,Pos,G,GS


## Load in Large Table to start -- Player ID Reverse Lookup

In [23]:
# * Batters
def identify_home_away_batters(full_game_df):
    home_batters = full_game_df.loc[full_game_df['inning_topbot'] == 'Bot'].batter.unique().tolist()
    away_batters = full_game_df.loc[full_game_df['inning_topbot'] == 'Top'].batter.unique().tolist()
    return home_batters, away_batters

In [24]:
identify_home_away_batters(gather_game_info('2023-04-15', 'CHC'))

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


([444482,
  518792,
  678246,
  681546,
  502110,
  571970,
  518692,
  605141,
  605131,
  500743],
 [])